# Task 2:
### Interactive QA Bot Interface
#### Problem Statement:
#### Develop an interactive interface for the QA bot from Part 1, allowing users to input queries and retrieve answers in real time. The interface should enable users to upload documents and ask questions based on the content of the uploaded document.
Task Requirements:
1. Build a simple frontend interface using Streamlit or Gradio, allowing users to
upload PDF documents and ask questions.
2. Integrate the backend from Part 1 to process the PDF, store document embeddings,
and provide real-time answers to user queries.
3. Ensure that the system can handle multiple queries efficiently and provide accurate,
contextually relevant responses.
4. Allow users to see the retrieved document segments alongside the generated
answer.


In [ ]:
# Setup and Requirements
!pip install -q streamlit PyPDF2 pinecone-client cohere transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 M

In [ ]:
import os
!wget -q -O - ipv4.icanhazip.com # xx.xxx.xxx.xxx    this ipv4 to be used as password for npx localtunnel port 8501

In [ ]:
%%writefile app.py
# import libraries
import PyPDF2
import streamlit as st
import pinecone
import cohere
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Extract pdf
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text

# Initialize pinecone
import os
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key="axxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

# Reset index
pc.delete_index('rag')

# Create index
if 'rag' not in pc.list_indexes().names():
          pc.create_index(
              name='rag',
              dimension=384,
              metric='cosine',
              spec=ServerlessSpec(
                  cloud='aws',
                  region='us-east-1'
              )
          )

# Vector Database Setup
index = pc.Index('rag')

# Initialize Cohere for text generation (alternatively, GPT-3/4 API can be used)
cohere_client = cohere.Client(api_key="bxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

# Load a pre-trained embedding model from Hugging Face (e.g., sentence-transformers)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # Average pooling

    # Convert to numpy array and cast to float32
    embeddings_np = embeddings.numpy().astype(np.float32)

    # L2 normalization (make sure the norm of the vector is 1)
    norms = np.linalg.norm(embeddings_np, axis=1, keepdims=True)
    normalized_embeddings = embeddings_np / norms  # Apply L2 normalization

    return normalized_embeddings

# Streamlit App
st.title("Interactive QA Bot with Document Upload")

# Step 1: Upload PDF
uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")
if uploaded_file:
    document_text = extract_text_from_pdf(uploaded_file)
    st.write("Document uploaded successfully!")

    # Step 2: Generate embeddings and store in Pinecone
    document_segments = document_text.split(". ")  # Split the document into sentences
    # Insert documents into Pinecone with their embeddings
    for i, segment in enumerate(document_segments):
          embedding = generate_embeddings([segment])[0].tolist()
          index.upsert([(f"doc_{i}", embedding, {"text": segment})])  # Store the embedding in Pinecone

    st.write(f"Stored {len(document_segments)} document segments in Pinecone.")

# Step 3: Query Input
query = st.text_input("Ask a question based on the document:")
if query and uploaded_file:
    # Retrieve relevant document segments from Pinecone
    def retrieve_relevant_docs(query, top_k=3):
        query_embedding = generate_embeddings([query])[0].tolist()

        results = index.query(vector=[query_embedding], top_k=top_k)
        relevant_docs = []
        if 'matches' in results and results['matches']:
            for match in results['matches']:
                doc_id = match['id']
                doc_index = int(doc_id.split("_")[1])  # Assuming "doc_X" format
                relevant_docs.append(document_segments[doc_index])
        else:
              print("No matches found in the query results.")
        return relevant_docs

    # Generate the answer using Cohere or any generative model
    def generate_answer(query, relevant_docs):
        context = "\n".join(relevant_docs)
        prompt = f"Question: {query}\n\nContext:\n{context}\n\nAnswer:"
        response = cohere_client.generate(
            model="command-nightly",
            prompt=prompt,
            max_tokens=700,
            temperature=0.5
        )
        return response.generations[0].text.strip()

    # QA Bot Function
    def qa_bot(query):
        # Retrieve relevant documents based on the query
        relevant_docs = retrieve_relevant_docs(query)

        # Generate a coherent answer using Cohere
        answer = generate_answer(query, relevant_docs)
        return answer, relevant_docs

    # Display the generated answer
    answer, relevant_docs = qa_bot(query)
    st.write("Answer:")
    st.write(answer)
    st.write("Relevant Document Segments:")
    for doc in relevant_docs:
        st.write(f"- {doc}")

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.204.10.61:8501

your url is: https://early-tools-type.loca.lt
  Stopping...
^C
